In [1]:
import pandas as pd
import os
import boto3
import pyarrow.parquet as pq
from io import BytesIO
import numpy as np
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

#print(S3_BUCKET_NAME)

# Check if environment variables are set
if not all([AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, S3_BUCKET_NAME]):
    raise ValueError("AWS credentials or S3 bucket name not set")

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

def read_and_merge_parquet_files(s3_bucket):
    try:
        # List objects in the S3 bucket
        objects = s3.list_objects(Bucket=s3_bucket)

        # Process the objects and create a DataFrame
        dfs = []
        for obj in objects.get('Contents', []):
            # Your logic to read and process Parquet files goes here
            # For example, using pyarrow to read Parquet file
            response = s3.get_object(Bucket=s3_bucket, Key=obj['Key'])
            parquet_file = BytesIO(response['Body'].read())
            df = pq.read_table(parquet_file).to_pandas()
            dfs.append(df)

        # Merge the dataframes
        if dfs:
            merged_df = pd.concat(dfs, ignore_index=True)
            return merged_df
        else:
            print("No Parquet files found in the S3 bucket.")
            return None
    
    except Exception as e:
        print(f"Error: {e}")
        return None

# Call the function
merged_data = read_and_merge_parquet_files(S3_BUCKET_NAME)

# Print the result if it's not None
if merged_data is not None:
    print(merged_data)
else:
    print("Error occurred during processing.")


       class  aa_000 ab_000 ac_000 ad_000 ae_000 af_000 ag_000  ag_001  \
0        pos   73930      0     na     na      0      0      0     582   
1        pos  354598      0      0     na      0      0      0       0   
2        pos  189008      4   1130     na      0      0   8252  151670   
3        pos      26     na      8      8      0      0      0       0   
4        pos  356414     na   3326    964      0      0      0     684   
...      ...     ...    ...    ...    ...    ...    ...    ...     ...   
108494   neg   47332     na    660    652      0      0      0       0   
108495   neg  104910     na    224      0      0      0      0       0   
108496   neg     546      0     42     20      0      0      0       0   
108497   neg   43506     na    490    378      0      0      0       0   
108498   neg     214     na     24     24      0      0      0       0   

        ag_002  ...  ee_003   ee_004   ee_005   ee_006  ee_007  ee_008 ee_009  \
0        77060  ...  147600   

In [5]:
print(merged_data)

       class  aa_000 ab_000 ac_000 ad_000 ae_000 af_000 ag_000  ag_001  \
0        pos   73930      0     na     na      0      0      0     582   
1        pos  354598      0      0     na      0      0      0       0   
2        pos  189008      4   1130     na      0      0   8252  151670   
3        pos      26     na      8      8      0      0      0       0   
4        pos  356414     na   3326    964      0      0      0     684   
...      ...     ...    ...    ...    ...    ...    ...    ...     ...   
108494   neg   47332     na    660    652      0      0      0       0   
108495   neg  104910     na    224      0      0      0      0       0   
108496   neg     546      0     42     20      0      0      0       0   
108497   neg   43506     na    490    378      0      0      0       0   
108498   neg     214     na     24     24      0      0      0       0   

        ag_002  ...  ee_003   ee_004   ee_005   ee_006  ee_007  ee_008 ee_009  \
0        77060  ...  147600   

In [6]:
merged_data.drop("partition_0",axis=1,inplace=True)

In [7]:
unique_values_column11 = merged_data['class'].value_counts()

In [8]:
import numpy as np

In [9]:

merged_data.replace('na', np.nan, inplace=True)

In [10]:
merged_data['class'] = merged_data['class'].replace('pos', 1).replace(['neg'], 0)


In [11]:
x = [i for i in merged_data.columns]
filtered_list = [item for item in x if item not in ("class")]


In [12]:
for column in filtered_list:
    merged_data[column] = pd.to_numeric(merged_data[column], errors='coerce')


In [13]:
correlation_matrix = merged_data.corr()


highly_correlated_pairs = []
considered_columns = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        colname1 = correlation_matrix.columns[i]
        colname2 = correlation_matrix.columns[j]
        
        
        if colname1 != colname2:
            
            if colname1 not in considered_columns and colname2 not in considered_columns:
                if abs(correlation_matrix.iloc[i, j]) > 0.85:
                    pair = (colname1, colname2)
                    highly_correlated_pairs.append(pair)
                    considered_columns.add(colname1)
                    considered_columns.add(colname2)

unique_columns = list(set(column for pair in highly_correlated_pairs for column in pair))

print("List of unique columns from highly correlated pairs:", unique_columns)

List of unique columns from highly correlated pairs: ['cb_000', 'ag_002', 'bx_000', 'ba_003', 'cs_000', 'bo_000', 'ay_002', 'bu_000', 'bq_000', 'bv_000', 'cn_005', 'al_000', 'ak_000', 'an_000', 'bi_000', 'by_000', 'cn_001', 'dl_000', 'aj_000', 'ag_003', 'ec_00', 'ba_000', 'cq_000', 'cn_004', 'ap_000', 'cs_002', 'cv_000', 'aq_000', 'do_000', 'ba_005', 'ds_000', 'ag_007', 'ah_000', 'az_005', 'ag_006', 'am_0', 'bp_000', 'bt_000', 'ca_000', 'dm_000', 'ee_003', 'bh_000', 'cf_000', 'ad_000', 'bb_000', 'bn_000', 'ee_004', 'ay_003', 'ba_002', 'ag_004', 'ba_006', 'cc_000', 'ba_004', 'cn_003', 'cn_002', 'cs_005', 'dp_000', 'bj_000', 'ee_002', 'ag_005', 'ee_000', 'ao_000', 'bg_000', 'aa_000', 'bm_000', 'bl_000', 'dt_000', 'dc_000', 'ci_000', 'ba_001']


In [14]:
merged_data.drop(unique_columns,axis=1,inplace=True)

In [15]:


y = merged_data['class']  


X_train, X_test, y_train, y_test = train_test_split(merged_data.drop('class', axis=1) , y, test_size=0.2, random_state=42)


In [16]:
numeric_features=X_train.columns

In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=6))  # You can adjust the number of components as needed
        ]), numeric_features)
    ])


In [18]:
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])


X_train = pipeline.fit_transform(X_train)
X_test= pipeline.transform(X_test)

In [19]:
import os
import matplotlib.pyplot as plt

# Create a folder named 'Aux_data' if it doesn't exist
plots_folder = 'Aux_data'
os.makedirs(plots_folder, exist_ok=True)

num_columns = X_train.shape[1]

for i in range(num_columns):
    # Create a new figure for each feature
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot the histogram for the current feature
    ax.hist(X_train[:, i], bins=10, color='blue', alpha=0.7)
    ax.set_title(f'Feature {i + 1}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')

    # Save the plot for the current feature
    plot_filename = os.path.join(plots_folder, f'feature_{i + 1}_histogram.png')
    plt.savefig(plot_filename)

    # Close the current figure to avoid overlapping plots
    plt.close()


In [20]:

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


In [21]:
len(X_train)

168906

In [22]:
len(y_train)

168906

In [23]:
len(y_test)

21700

In [24]:
len(X_test)

21700

In [25]:
import mlflow
import mlflow.sklearn
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score

# Assuming X_train, X_test, y_train, y_test are defined before this point

def objective(trial):
    dt_params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 32),
        'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
    }

    dt_model = DecisionTreeClassifier(**dt_params)
    dt_model.fit(X_train, y_train)

    y_pred = dt_model.predict(X_test)

    recall = recall_score(y_test, y_pred)

    # Log parameters and metrics for each trial
    with mlflow.start_run(nested=True):
        mlflow.log_params(dt_params)
        mlflow.log_metric("recall", recall)

    return recall

mlflow.set_experiment("decision_tree_hyperparameter_tuning_recall")

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# End the active run before logging the best parameters and model
mlflow.end_run()

# Log the best parameters and model
with mlflow.start_run():
    best_params = study.best_params
    mlflow.log_params(best_params)

    best_dt_model = DecisionTreeClassifier(**best_params)
    best_dt_model.fit(X_train, y_train)

    # Log the best model
    mlflow.sklearn.log_model(best_dt_model, "decision_tree_model")



c:\Users\awscl\.conda\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-12-19 10:11:17,376] A new study created in memory with name: no-name-13528cb9-2149-40b7-b991-e0bf5f635174
[I 2023-12-19 10:11:18,153] Trial 0 finished with value: 0.8387096774193549 and parameters: {'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 0.6298897525989567, 'min_samples_leaf': 0.29616988632326247, 'max_features': 0.3391537552508472}. Best is trial 0 with value: 0.8387096774193549.
[I 2023-12-19 10:11:18,430] Trial 1 finished with value: 0.9219015280135824 and parameters: {'criterion': 'entropy', 'max_depth': 32, 'min_samples_split': 0.7872839475171833, 'min_samples_leaf': 0.4555988309691503, 'max_features': 0.6279799281724814}. Best is trial 1 with value: 0.9219015280135824.
[I 2023-12-19 10:11:18,635] Trial 2

In [25]:
import mlflow
import mlflow.xgboost
import optuna
import xgboost as xgb
from sklearn.metrics import recall_score

# Assuming X_train, X_test, y_train, y_test are defined before this point

def objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
    }

    xgb_model = xgb.XGBClassifier(**xgb_params)
    xgb_model.fit(X_train, y_train)

    y_pred = xgb_model.predict(X_test)

    recall = recall_score(y_test, y_pred)

    # Log parameters and metrics for each trial
    with mlflow.start_run(nested=True):
        mlflow.log_params(xgb_params)
        mlflow.log_metric("recall", recall)

    return recall

mlflow.set_experiment("xgboost_hyperparameter_tuning_recall")

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# End the active run before logging the best parameters and model
mlflow.end_run()

# Log the best parameters and model
with mlflow.start_run():
    best_params = study.best_params
    mlflow.log_params(best_params)

    best_xgb_model = xgb.XGBClassifier(**best_params)
    best_xgb_model.fit(X_train, y_train)

    # Log the best model
    mlflow.xgboost.log_model(best_xgb_model, "xgboost_model")


[I 2023-12-19 11:43:51,471] A new study created in memory with name: no-name-ca5bf149-e38f-4805-9a93-7d792048b22a
[I 2023-12-19 11:43:53,449] Trial 0 finished with value: 0.9235993208828522 and parameters: {'booster': 'gbtree', 'max_depth': 1, 'learning_rate': 0.012232774138634394, 'subsample': 0.6160924570671653, 'colsample_bytree': 0.9834917317999247, 'gamma': 0.7839711848737682}. Best is trial 0 with value: 0.9235993208828522.
[I 2023-12-19 11:45:22,074] Trial 1 finished with value: 0.9473684210526315 and parameters: {'booster': 'dart', 'max_depth': 6, 'learning_rate': 0.014685901707503474, 'subsample': 0.8153224228096361, 'colsample_bytree': 0.570048748706728, 'gamma': 0.6958991560970811}. Best is trial 1 with value: 0.9473684210526315.
[I 2023-12-19 11:45:25,352] Trial 2 finished with value: 0.9405772495755518 and parameters: {'booster': 'gbtree', 'max_depth': 9, 'learning_rate': 0.021084070517431924, 'subsample': 0.8380909165622664, 'colsample_bytree': 0.2367987938585752, 'gamma'

In [24]:
import mlflow
import mlflow.sklearn
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score


def objective(trial):
    rf_params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 32),
        'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
    }

    rf_model = RandomForestClassifier(**rf_params)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)

    recall = recall_score(y_test, y_pred)

    # Log parameters and metrics for each trial
    with mlflow.start_run(nested=True):
        mlflow.log_params(rf_params)
        mlflow.log_metric("recall", recall)

    return recall

mlflow.set_experiment("random_forest_hyperparameter_tuning_recall")

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# End the active run before logging the best parameters and model
mlflow.end_run()

# Log the best parameters and model
with mlflow.start_run():
    best_params = study.best_params
    mlflow.log_params(best_params)

    best_rf_model = RandomForestClassifier(**best_params)
    best_rf_model.fit(X_train, y_train)

    # Log the best model
    mlflow.sklearn.log_model(best_rf_model, "random_forest_model")


c:\Users\awscl\.conda\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023/12/19 11:33:12 INFO mlflow.tracking.fluent: Experiment with name 'random_forest_hyperparameter_tuning_recall' does not exist. Creating a new experiment.
[I 2023-12-19 11:33:12,778] A new study created in memory with name: no-name-97100367-f9e5-4c8f-82f4-94d6dd2b4fc8
[I 2023-12-19 11:33:17,692] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 147, 'max_depth': 24, 'min_samples_split': 0.7695160042486281, 'min_samples_leaf': 0.20387740751640715, 'max_features': 0.2727339022357106}. Best is trial 0 with value: 0.0.
[I 2023-12-19 11:33:19,242] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 44, 'max_depth': 20, 'min_samples_split': 0.9393151702847801, 'min_samples_leaf': 0.3294236045743407, 'max_featur